In [23]:
import sqlite3
import json

# Funkcja łącząca się z bazą SQLite lub tworząca ją jeśli nie istnieje
def connect_sqlite(db_file="test.db"):
    return sqlite3.connect(db_file)

# Funkcja tworząca tabele w bazie, jeśli jeszcze nie istnieją
def create_tables_sqlite(conn):
    c = conn.cursor()
    # Tworzy tabelę 'mieszkanie' z kolumnami: id, adres i powierzchnia
    c.execute('''
        CREATE TABLE IF NOT EXISTS mieszkanie (
            id_mieszkania INTEGER PRIMARY KEY,
            adres TEXT,
            powierzchnia REAL
        )
    ''')
    # Tworzy tabelę 'wlasciciel' z danymi właściciela mieszkania
    c.execute('''
        CREATE TABLE IF NOT EXISTS wlasciciel (
            id_wlasciciela INTEGER PRIMARY KEY,
            imie TEXT,
            nazwisko TEXT,
            telefon TEXT,
            email TEXT
        )
    ''')
    # Tworzy tabelę 'licznik' zawierającą liczniki przypisane do mieszkań
    c.execute('''
        CREATE TABLE IF NOT EXISTS licznik (
            id_licznika INTEGER PRIMARY KEY,
            typ TEXT,
            id_mieszkania INTEGER,
            FOREIGN KEY(id_mieszkania) REFERENCES mieszkanie(id_mieszkania)
        )
    ''')
    # Tworzy tabelę 'pracownik' z danymi pracowników (np. którzy wykonują pomiary)
    c.execute('''
        CREATE TABLE IF NOT EXISTS pracownik (
            id_pracownika INTEGER PRIMARY KEY,
            imie TEXT,
            nazwisko TEXT
        )
    ''')
    # Tworzy tabelę 'pomiar' przechowującą pomiary liczników z powiązaniem do pracownika i licznika
    c.execute('''
        CREATE TABLE IF NOT EXISTS pomiar (
            id_pomiaru INTEGER PRIMARY KEY,
            id_licznika INTEGER,
            wartosc_pomiaru REAL,
            data_pomiaru TEXT,
            id_pracownika INTEGER,
            FOREIGN KEY(id_licznika) REFERENCES licznik(id_licznika),
            FOREIGN KEY(id_pracownika) REFERENCES pracownik(id_pracownika)
        )
    ''')
    # Zatwierdza zmiany w bazie danych
    conn.commit()

# Funkcje do wstawiania danych do odpowiednich tabel

def insert_mieszkanie(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO mieszkanie VALUES (?, ?, ?)', (data['id_mieszkania'], data['adres'], data['powierzchnia']))
    conn.commit()

def insert_wlasciciel(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO wlasciciel VALUES (?, ?, ?, ?, ?)', (data['id_wlasciciela'], data['imie'], data['nazwisko'], data['telefon'], data['email']))
    conn.commit()

def insert_licznik(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO licznik VALUES (?, ?, ?)', (data['id_licznika'], data['typ'], data['id_mieszkania']))
    conn.commit()

def insert_pracownik(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO pracownik VALUES (?, ?, ?)', (data['id_pracownika'], data['imie'], data['nazwisko']))
    conn.commit()

def insert_pomiar(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO pomiar VALUES (?, ?, ?, ?, ?)', (data['id_pomiaru'], data['id_licznika'], data['wartosc_pomiaru'], data['data_pomiaru'], data['id_pracownika']))
    conn.commit()

# Funkcja czyszcząca zawartość wszystkich tabel w bazie danych
def clear_db(conn):
    c = conn.cursor()
    tables = ['pomiar', 'pracownik', 'licznik', 'wlasciciel', 'mieszkanie']
    for t in tables:
        c.execute(f'DELETE FROM {t}')  # Usuwa wszystkie rekordy z danej tabeli
    conn.commit()


In [4]:
import sqlite3
import json

# Funkcja łącząca się z bazą SQLite lub tworząca nową bazę, jeśli plik nie istnieje
# Argument db_file to nazwa pliku bazy danych (domyślnie "test.db")
def connect_sqlite(db_file="test.db"):
    return sqlite3.connect(db_file)

# Funkcja tworząca tabele w bazie danych, jeśli jeszcze ich nie ma
# Przyjmuje jako argument aktywne połączenie do bazy (conn)
def create_tables_sqlite(conn):
    c = conn.cursor()  # Tworzy obiekt kursora, który pozwala wykonywać zapytania SQL
    
    # Tworzy tabelę 'mieszkanie' z kolumnami: id_mieszkania (klucz główny), adres i powierzchnia mieszkania
    c.execute('''
        CREATE TABLE IF NOT EXISTS mieszkanie (
            id_mieszkania INTEGER PRIMARY KEY,
            adres TEXT,
            powierzchnia REAL
        )
    ''')
    
    # Tworzy tabelę 'wlasciciel' z danymi właściciela mieszkania: id, imię, nazwisko, telefon i email
    c.execute('''
        CREATE TABLE IF NOT EXISTS wlasciciel (
            id_wlasciciela INTEGER PRIMARY KEY,
            imie TEXT,
            nazwisko TEXT,
            telefon TEXT,
            email TEXT
        )
    ''')
    
    # Tworzy tabelę 'licznik', która przechowuje liczniki (np. wodomierze) przypisane do mieszkań
    # id_licznika to klucz główny, id_mieszkania to klucz obcy wskazujący mieszkanie, do którego licznik należy
    c.execute('''
        CREATE TABLE IF NOT EXISTS licznik (
            id_licznika INTEGER PRIMARY KEY,
            typ TEXT,
            id_mieszkania INTEGER,
            FOREIGN KEY(id_mieszkania) REFERENCES mieszkanie(id_mieszkania)
        )
    ''')
    
    # Tworzy tabelę 'pracownik', z danymi pracowników wykonujących pomiary lub inne czynności
    c.execute('''
        CREATE TABLE IF NOT EXISTS pracownik (
            id_pracownika INTEGER PRIMARY KEY,
            imie TEXT,
            nazwisko TEXT
        )
    ''')
    
    # Tworzy tabelę 'pomiar', która przechowuje pomiary liczników, wraz z datą, wartością i informacją o pracowniku
    # id_licznika i id_pracownika to klucze obce do tabel licznik i pracownik
    c.execute('''
        CREATE TABLE IF NOT EXISTS pomiar (
            id_pomiaru INTEGER PRIMARY KEY,
            id_licznika INTEGER,
            wartosc_pomiaru REAL,
            data_pomiaru TEXT,
            id_pracownika INTEGER,
            FOREIGN KEY(id_licznika) REFERENCES licznik(id_licznika),
            FOREIGN KEY(id_pracownika) REFERENCES pracownik(id_pracownika)
        )
    ''')
    
    # Zapisuje wszystkie powyższe zmiany w bazie danych
    conn.commit()

# Funkcja do wstawiania nowego rekordu do tabeli 'mieszkanie'
# Argument 'data' to słownik z kluczami odpowiadającymi kolumnom tabeli
def insert_mieszkanie(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO mieszkanie VALUES (?, ?, ?)', (data['id_mieszkania'], data['adres'], data['powierzchnia']))
    conn.commit()

# Funkcja do wstawiania nowego rekordu do tabeli 'wlasciciel'
def insert_wlasciciel(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO wlasciciel VALUES (?, ?, ?, ?, ?)', 
              (data['id_wlasciciela'], data['imie'], data['nazwisko'], data['telefon'], data['email']))
    conn.commit()

# Funkcja do wstawiania nowego rekordu do tabeli 'licznik'
def insert_licznik(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO licznik VALUES (?, ?, ?)', (data['id_licznika'], data['typ'], data['id_mieszkania']))
    conn.commit()

# Funkcja do wstawiania nowego rekordu do tabeli 'pracownik'
def insert_pracownik(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO pracownik VALUES (?, ?, ?)', (data['id_pracownika'], data['imie'], data['nazwisko']))
    conn.commit()

# Funkcja do wstawiania nowego rekordu do tabeli 'pomiar'
def insert_pomiar(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO pomiar VALUES (?, ?, ?, ?, ?)', 
              (data['id_pomiaru'], data['id_licznika'], data['wartosc_pomiaru'], data['data_pomiaru'], data['id_pracownika']))
    conn.commit()

# Funkcja usuwająca wszystkie rekordy ze wszystkich tabel w bazie danych
# Przydatna np. do czyszczenia bazy przed załadowaniem nowych danych
def clear_db(conn):
    c = conn.cursor()
    tables = ['pomiar', 'pracownik', 'licznik', 'wlasciciel', 'mieszkanie']
    for t in tables:
        c.execute(f'DELETE FROM {t}')  # Usuwa wszystkie wpisy w danej tabeli
    conn.commit()


Wygenerowano dane i zapisano do plików JSON i CSV.


In [9]:
import psycopg

# Funkcja łącząca się z bazą PostgreSQL
# Przyjmuje nazwę bazy, użytkownika, hasło, host i port (domyślnie localhost:5432)
def connect_postgres(dbname, user, password, host='localhost', port=5432):
    return psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Funkcja tworząca potrzebne tabele w bazie PostgreSQL, jeśli jeszcze nie istnieją
# Korzysta z SERIAL jako autoinkrementującego klucza głównego
def create_tables_postgres(conn):
    c = conn.cursor()
    
    # Tabela mieszkanie z kolumnami id, adres i powierzchnia
    c.execute('''
        CREATE TABLE IF NOT EXISTS mieszkanie (
            id_mieszkania SERIAL PRIMARY KEY,
            adres TEXT,
            powierzchnia REAL
        )
    ''')
    
    # Tabela wlasciciel z danymi właściciela
    c.execute('''
        CREATE TABLE IF NOT EXISTS wlasciciel (
            id_wlasciciela SERIAL PRIMARY KEY,
            imie TEXT,
            nazwisko TEXT,
            telefon TEXT,
            email TEXT
        )
    ''')
    
    # Tabela licznik, powiązana kluczem obcym z tabelą mieszkanie
    c.execute('''
        CREATE TABLE IF NOT EXISTS licznik (
            id_licznika SERIAL PRIMARY KEY,
            typ TEXT,
            id_mieszkania INTEGER REFERENCES mieszkanie(id_mieszkania)
        )
    ''')
    
    # Tabela pracownik z danymi pracowników
    c.execute('''
        CREATE TABLE IF NOT EXISTS pracownik (
            id_pracownika SERIAL PRIMARY KEY,
            imie TEXT,
            nazwisko TEXT
        )
    ''')
    
    # Tabela pomiar z pomiarami liczników
    # Powiązana z licznikami i pracownikami przez klucze obce
    c.execute('''
        CREATE TABLE IF NOT EXISTS pomiar (
            id_pomiaru SERIAL PRIMARY KEY,
            id_licznika INTEGER REFERENCES licznik(id_licznika),
            wartosc_pomiaru REAL,
            data_pomiaru TIMESTAMP,
            id_pracownika INTEGER REFERENCES pracownik(id_pracownika)
        )
    ''')
    
    conn.commit()

# Funkcja dodająca nowy rekord do tabeli mieszkanie
# Dane przekazywane jako słownik z kluczami: id_mieszkania, adres, powierzchnia
def insert_mieszkanie_pg(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO mieszkanie (id_mieszkania, adres, powierzchnia) VALUES (%s, %s, %s)', 
              (data['id_mieszkania'], data['adres'], data['powierzchnia']))
    conn.commit()

# Funkcja dodająca nowy rekord do tabeli wlasciciel
def insert_wlasciciel_pg(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO wlasciciel (id_wlasciciela, imie, nazwisko, telefon, email) VALUES (%s, %s, %s, %s, %s)', 
              (data['id_wlasciciela'], data['imie'], data['nazwisko'], data['telefon'], data['email']))
    conn.commit()

# Funkcja dodająca nowy rekord do tabeli licznik
def insert_licznik_pg(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO licznik (id_licznika, typ, id_mieszkania) VALUES (%s, %s, %s)', 
              (data['id_licznika'], data['typ'], data['id_mieszkania']))
    conn.commit()

# Funkcja dodająca nowy rekord do tabeli pracownik
def insert_pracownik(conn, data):
    c = conn.cursor()
    c.execute('INSERT INTO pracownik (id_pracownika, imie, nazwisko) VALUES (%s, %s, %s)', 
              (data['id_pracownika'], data['imie'], data['nazwisko']))
    conn.commit()

# Funkcja dodająca nowy rekord do tabeli pomiar
def insert_pomiar(conn, data):
    c = conn.cursor()
    # Uwaga: w zapytaniu jest 5 miejsc na wartości (%s), ale podano 4 wartości — powinno być 5
    c.execute('INSERT INTO pomiar (id_pomiaru, id_licznika, wartosc_pomiaru, data_pomiaru, id_pracownika) VALUES (%s, %s, %s, %s, %s)', 
              (data['id_pomiaru'], data['id_licznika'], data['wartosc_pomiaru'], data['data_pomiaru'], data['id_pracownika']))
    conn.commit()

# Funkcja czyści wszystkie dane z tabel w bazie, usuwając wszystkie rekordy
# Używa TRUNCATE z CASCADE, aby usunąć powiązane dane w zależnościach
def clear_db_postgres(conn):
    c = conn.cursor()
    tables = ['pomiar', 'pracownik', 'licznik', 'wlasciciel', 'mieszkanie']
    for t in tables:
        c.execute(f'TRUNCATE TABLE {t} CASCADE')
    conn.commit()


In [15]:
import csv

# Funkcja ładująca dane z pliku CSV do tabeli SQLite
# conn - połączenie z bazą danych SQLite
# csv_file - ścieżka do pliku CSV
# insert_func - funkcja wstawiająca pojedynczy rekord do odpowiedniej tabeli
def load_csv_to_sqlite_table(conn, csv_file, insert_func):
    # Otwieramy plik CSV z kodowaniem UTF-8
    with open(csv_file, newline='', encoding='utf-8') as f:
        # Tworzymy czytnik, który odczytuje wiersze jako słowniki (klucz = nazwa kolumny)
        reader = csv.DictReader(f)
        # Dla każdego wiersza wywołujemy funkcję insertującą dane do bazy
        for row in reader:
            insert_func(conn, row)






## ✅ Zadanie 1: Raportowanie danych z bazy SQLite

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

# Funkcja generująca rozszerzony raport na podstawie danych z bazy SQLite
def generate_extended_report_sqlite(db_file='test.db'):
    # Nawiązanie połączenia z bazą SQLite
    conn = sqlite3.connect(db_file)

    # Wczytanie całej tabeli 'pomiar' do DataFrame pandas
    df_pomiar = pd.read_sql_query("SELECT * FROM pomiar", conn)
    
    # Konwersja kolumny z datą pomiaru na typ datetime (do wygodniejszej analizy i wykresów)
    df_pomiar['data_pomiaru'] = pd.to_datetime(df_pomiar['data_pomiaru'])

    # Wyświetlenie liczby wszystkich pomiarów w danych
    print("📌 Liczba pomiarów:", len(df_pomiar))
    # Wyświetlenie podstawowych statystyk opisowych dla kolumny z wartością pomiaru
    print(df_pomiar['wartosc_pomiaru'].describe())

    # Wykres 1: Pomiary wykonane przez pracownika o ID 401
    df_plot1 = df_pomiar[df_pomiar['id_pracownika'] == 401].copy()
    # Posortowanie danych wg daty pomiaru rosnąco, aby wykres miał sensowną oś czasu
    df_plot1 = df_plot1.sort_values('data_pomiaru')

    # Sprawdzenie, czy dane nie są puste (nie rysujemy wykresu, jeśli brak danych)
    if not df_plot1.empty:
        plt.figure(figsize=(10, 4))
        # Rysowanie wykresu liniowego z punktami
        plt.plot(df_plot1['data_pomiaru'], df_plot1['wartosc_pomiaru'], marker='o')
        plt.title("Pomiar wartości – pracownik 401")
        plt.xlabel("Data")
        plt.ylabel("Wartość")
        plt.grid(True)
        plt.show()

    # Wykres 3: Porównanie średnich wartości pomiarów dla 5 liczników z największą liczbą pomiarów
    # Wybieramy ID 5 liczników, które mają najwięcej wpisów w danych
    top5 = df_pomiar['id_licznika'].value_counts().nlargest(5).index
    # Filtrowanie danych tylko dla tych 5 liczników
    df_avg = df_pomiar[df_pomiar['id_licznika'].isin(top5)]
    # Grupowanie danych wg ID licznika i wyliczenie średniej wartości pomiaru
    grouped = df_avg.groupby('id_licznika')['wartosc_pomiaru'].mean().reset_index()

    plt.figure(figsize=(8, 5))
    # Wykres słupkowy pokazujący średnią wartość pomiaru dla każdego z 5 liczników
    plt.bar(grouped['id_licznika'].astype(str), grouped['wartosc_pomiaru'], color='orange')
    plt.title(" Średnia wartość pomiaru dla 5 liczników")
    plt.xlabel("ID licznika")
    plt.ylabel("Średnia wartość")
    plt.grid(axis='y')
    plt.show()


## ✅ Zadanie 2: Filtrowanie / wyszukiwanie danych bez SQL

In [11]:
import sqlite3
import pandas as pd

# Funkcja łącząca się z bazą SQLite
def connect(db_file='test.db'):
    return sqlite3.connect(db_file)

# 1. Funkcja wyszukująca pomiary według nazwiska właściciela mieszkania
def find_by_owner_lastname(nazwisko, db_file='test.db'):
    conn = connect(db_file)
    query = """
    SELECT wlasciciel.imie, wlasciciel.nazwisko, mieszkanie.adres, licznik.typ, pomiar.wartosc_pomiaru, pomiar.data_pomiaru
    FROM wlasciciel
    -- Łączenie z tabelą licznik po id właściciela i id mieszkania (tu jest błąd w zapytaniu, powinno być inaczej)
    JOIN licznik ON wlasciciel.id_wlasciciela = licznik.id_mieszkania
    JOIN mieszkanie ON licznik.id_mieszkania = mieszkanie.id_mieszkania
    JOIN pomiar ON licznik.id_licznika = pomiar.id_licznika
    WHERE wlasciciel.nazwisko LIKE ?
    """
    # Wykonanie zapytania z parametrem nazwiska, używając LIKE z procentami dla dopasowania fragmentu
    df = pd.read_sql_query(query, conn, params=(f"%{nazwisko}%",))
    conn.close()
    return df

# 2. Funkcja wyszukująca pomiary według fragmentu adresu mieszkania
def find_by_address(adres_fragment, db_file='test.db'):
    conn = connect(db_file)
    query = """
    SELECT mieszkanie.adres, licznik.typ, pomiar.wartosc_pomiaru, pomiar.data_pomiaru
    FROM mieszkanie
    JOIN licznik ON mieszkanie.id_mieszkania = licznik.id_mieszkania
    JOIN pomiar ON licznik.id_licznika = pomiar.id_licznika
    WHERE mieszkanie.adres LIKE ?
    """
    df = pd.read_sql_query(query, conn, params=(f"%{adres_fragment}%",))
    conn.close()
    return df

# 3. Funkcja wyszukująca pomiary w zadanym zakresie dat (start_date i end_date w formacie 'YYYY-MM-DD')
def find_by_date_range(start_date, end_date, db_file='test.db'):
    conn = connect(db_file)
    query = """
    SELECT * FROM pomiar
    WHERE data_pomiaru BETWEEN ? AND ?
    """
    df = pd.read_sql_query(query, conn, params=(start_date, end_date))
    conn.close()
    return df

# 4. Funkcja wyszukująca pomiary wykonane przez konkretnego pracownika wg jego ID
def find_by_worker(worker_id, db_file='test.db'):
    conn = connect(db_file)
    query = """
    SELECT pracownik.imie, pracownik.nazwisko, licznik.typ, pomiar.wartosc_pomiaru, pomiar.data_pomiaru
    FROM pracownik
    JOIN pomiar ON pracownik.id_pracownika = pomiar.id_pracownika
    JOIN licznik ON pomiar.id_licznika = licznik.id_licznika
    WHERE pracownik.id_pracownika = ?
    """
    df = pd.read_sql_query(query, conn, params=(worker_id,))
    conn.close()
    return df

# 5. Funkcja wyszukująca wszystkie pomiary konkretnego licznika po jego ID
def find_by_meter(meter_id, db_file='test.db'):
    conn = connect(db_file)
    query = """
    SELECT licznik.typ, mieszkanie.adres, pomiar.wartosc_pomiaru, pomiar.data_pomiaru
    FROM licznik
    JOIN mieszkanie ON licznik.id_mieszkania = mieszkanie.id_mieszkania
    JOIN pomiar ON licznik.id_licznika = pomiar.id_licznika
    WHERE licznik.id_licznika = ?
    """
    df = pd.read_sql_query(query, conn, params=(meter_id,))
    conn.close()
    return df
